In [1]:
# Standard imports
import cv2
import numpy as np 

# Read images
src = cv2.imread("images/smoke.jpg")
dst = cv2.imread("images/background.jpg")

# Create a rough mask around the airplane.
src_mask = np.zeros(src.shape, src.dtype)

# Resize
dst = cv2.resize(dst, (500, 300), interpolation = cv2.INTER_CUBIC)

poly = np.array([   [
          396.2770562770563,
          160.6060606060606
        ],
        [
          397.5757575757575,
          170.56277056277057
        ],
        [
          401.9047619047619,
          175.75757575757575
        ],
        [
          412.72727272727275,
          185.28138528138527
        ],
        [
          420.95238095238096,
          195.67099567099567
        ],
        [
          438.7012987012987,
          183.54978354978354
        ],
        [
          443.030303030303,
          167.09956709956708
        ],
        [
          444.7619047619047,
          154.978354978355
        ],
        [
          421.81818181818176,
          148.91774891774892
        ],
        [
          404.069264069264,
          153.67965367965368
        ] ], np.int32)
cv2.fillPoly(src_mask, [poly], (255, 255, 255)) #原图的mask
src_mask.shape, dst.shape

((425, 640, 3), (300, 500, 3))

In [19]:
# np.max(poly), np.min(poly)
# poly.shape, poly[:, 0], poly[:, 1]
print(np.max(poly[:, 0]), np.min(poly[:, 0]), np.max(poly[:, 1]), np.min(poly[:, 1]))
y_max, y_min, x_max, x_min = np.max(poly[:, 0]), np.min(poly[:, 0]), np.max(poly[:, 1]), np.min(poly[:, 1]) # mask最小方框的坐标
width = np.max(poly[:, 0]) - np.min(poly[:, 0]) # 最小方框的宽
height =  np.max(poly[:, 1]) - np.min(poly[:, 1]) # 最小方框的高 

# src_center = [(x_min+x_max) / 2, (y_min+y_max) / 2] # 原图中最小方框的中心坐标

# print(src_center, poly[:, 0] - src_center[0], poly[:, 1] - src_center[1])
# x_max, x_min, y_max, y_min, width, length

# poly_dst = np.zeros(poly.shape, poly.dtype)
# poly_dst[:, 0] = center[0] + poly[:, 0] - src_center[0] 
# poly_dst[:, 1] = center[1] + poly[:, 1] - src_center[1]
# poly_dst
# src 中的roi
src_roi = src[int(x_min): int(x_max), int(y_min): int(y_max)]
print(int(x_min), int(x_max), int(y_min), int(y_max))
# 这是 CENTER 所在的地方
center = (233, 133)  # center 是(y, x) cv2.imread 是(x, y), poly 是(y,x) x 是垂直方向， y 是水平方向

# dst 中的roi
dst_roi = dst[int(center[1] - height / 2): int(center[1] + height / 2), int(center[0] - width / 2): int(center[0] + width / 2)]

# src_mask 中的roi
mask_roi = src_mask[int(x_min): int(x_max), int(y_min): int(y_max)]
src.shape, src_roi.shape, dst_roi.shape,mask_roi.shape , src_mask.shape

444 396 195 148
148 195 396 444


((425, 640, 3), (47, 48, 3), (47, 48, 3), (47, 48, 3), (425, 640, 3))

In [20]:
# output_ORIGIN = cv2.add(src_roi, dst_roi, mask=mask_roi)
cv2.imwrite("images/src_roi.jpg", src_roi)
cv2.imwrite("images/dst_roi.jpg", dst_roi)
cv2.imwrite("images/mask_roi.jpg", mask_roi)

True

In [21]:
# Standard imports
import cv2
import numpy as np 

src_roi = cv2.imread("images/src_roi.jpg")
dst_roi = cv2.imread("images/dst_roi.jpg")
mask_roi = cv2.imread("images/mask_roi.jpg")



img2gray = cv2.cvtColor(mask_roi,cv2.COLOR_BGR2GRAY)      # 将图片灰度化
ret, mask = cv2.threshold(img2gray, 175, 255, cv2.THRESH_BINARY)#ret是阈值（175）mask是二值化图像， 虽然已经是二值化的图片了，但是没有这步效果很差，不知道为啥 
mask_inv = cv2.bitwise_not(mask)#获取把logo的区域取反 按位运算



# mask = cv2.cvtColor(mask_roi,cv2.COLOR_BGR2GRAY)
# mask_inv = cv2.bitwise_not(mask)





img1  =  cv2.bitwise_and(src_roi,src_roi,mask = mask)
img2  =  cv2.bitwise_and(dst_roi,dst_roi,mask = mask_inv)
add = cv2.add(img1,img2)#相加即可

cv2.imshow('img1',img1)
cv2.imshow('img2',img2)
cv2.imshow('dst',add)
cv2.waitKey(0)
cv2.destroyAllWindows()
# output_ORIGIN = cv2.add(src_roi, dst_roi, mask=mask_roi)
cv2.imwrite("images/img1.jpg", img1)
cv2.imwrite("images/img2.jpg", img2)
cv2.imwrite("images/add.jpg", add)

True

In [22]:
dst[int(center[1] - height / 2): int(center[1] + height / 2), int(center[0] - width / 2): int(center[0] + width / 2)] = add
cv2.imwrite("images/dst_add.jpg", dst)

True